#### Binary Floating Point Representation
Consider the problem of how to store a floating point number using 32 bit binary representation. The IEEE 754 standard defines the format for single precision floating point numbers.

In order to understand the format, we need to write some functions to convert between binary and decimal representations.  First, we write a function to convert any positive base-10 integer to a binary string.

In [2]:
def convert_int_to_binary(num):
    res = ""

    while num > 0:
        digit = num % 2
        # print(num, digit)
        res = str(digit) + res
        num = int(num / 2)

    return res

In [3]:
convert_int_to_binary(4096)

'1000000000000'

Next, we will write a function to convert a decimal number between 0 and 1 to a binary string.  We will use the following algorithm:

1. Multiply the number by 2
2. If the result is greater than or equal to 1, then the first digit in the binary representation is 1.  Subtract 1 from the result.
3. If the result is less than 1, then the first digit in the binary representation is 0.
4. Repeat steps 1-3 until the result is 0 or we have reached the desired precision.
5. Multiply the result by 2 and go to step 2.
6. If the result is 0, then we have reached the desired precision.  Otherwise, we have reached the desired precision and the next digit is 1.
7. Repeat steps 5-6 until the result is 0 or we have reached the desired precision.

In [4]:
def convert_fraction_to_binary(num, precision=40):
    res = ""
    iter_count = 0  
    while num > 0 and precision > 0:
        num = num * 2
        if num >= 1:
            res += "1"
            num -= 1
        else:
            res += "0"
        precision -= 1
        #print(f"num = {num}  res = {res}  iter_count = {iter_count}    precision = {precision}")
        iter_count += 1
    return res

In [5]:
convert_fraction_to_binary(0.125,35)

'001'

### IEEE 754 Floating Point Format for 64 bit numbers says its...

1 bit for the sign
11 bits for the exponent
52 bits for the mantissa

github copilot filled in all the bit information for the exponent and mantissa, how nice :D

In [6]:
def get_fp_binary_representation(n):
    # specify 48 decimal places of precision ... given that we will take 23 for the mantissa, this means we will have far more than we need
    p = 48

    # Step 1:  split the number into two parts - both strings
    front, back = str(n).split('.')

    # Step 2:  convert the part in front of the decimal to binary ... take the sign into account
    if int(front) < 0:
        sign = "-"
        i_front = -int(front)
    else:
        sign = ""
        i_front = int(front)

    if i_front == 0:
        front_bin = "0"
    else:
        front_bin = convert_int_to_binary(i_front)

    # Step 3:  convert the part after the decimal to binary
    divisor = 10**len(back)
    back_bin = convert_fraction_to_binary(float(back)/divisor, p)

    # Step 4:  add the strings together and print the result
    bin_result = sign + front_bin + "." + back_bin
    print(f"The binary representation of the {n} is {bin_result}")

    # Step 5:  Determine the exponent and mantissa
    if front_bin == "0":
        exponent = 0
        keep_going = True
        while keep_going:
            # print (back_bin[-exponent],exponent)
            if back_bin[-exponent] == "1":
                keep_going = False
                exponent = exponent + 1
            exponent = exponent - 1
        exponent = exponent - 1

        # print(exponent)
        back_bin = back_bin[-exponent:]
        mantissa_truncated = back_bin
    else:
        exponent = len(front_bin)-1
        mantissa = front_bin[1:] + back_bin
        mantissa_truncated = mantissa[0:52]

    true_result = sign + "1." + mantissa_truncated + " x 2^(" + str(exponent) + ")"
    print(f"The binary scientific notation representation is {true_result}")

    # Step 6:  Convert to 32-bit floating point representation
    if int(front) < 0:
        bit1 = "1"
    else:
        bit1 = "0"

    exp = int(exponent)+2047
    exp_binary_rep = convert_int_to_binary(exp)
    if len(exp_binary_rep) < 8:
        exp_binary_rep = "0" + exp_binary_rep

    if len(mantissa_truncated) < 52:
        mantissa_truncated = mantissa_truncated + (52-len(mantissa_truncated))*"0"

    if len(mantissa_truncated) > 52:
        mantissa_truncated = mantissa_truncated[0:52]

    res = bit1 + "|" + exp_binary_rep + "|" + mantissa_truncated
    return res

In [7]:
#fp_num = input("Enter your floating point value : \n")
fp_num = 3.14159265
result = get_fp_binary_representation(fp_num)
print(f"The IEEE 754 representation is {result}")

print()

fp_num = 3.14159266
result = get_fp_binary_representation(fp_num)
print(f"The IEEE 754 representation is {result}")

print()

fp_num = 1.25
result = get_fp_binary_representation(fp_num)
print(f"The IEEE 754 representation is {result}")

print()

fp_num = -1.25
result = get_fp_binary_representation(fp_num)
print(f"The IEEE 754 representation is {result}")

The binary representation of the 3.14159265 is 11.001001000011111101101010011110010001101010011110
The binary scientific notation representation is 1.1001001000011111101101010011110010001101010011110 x 2^(1)
The IEEE 754 representation is 0|100000000000|1001001000011111101101010011110010001101010011110000

The binary representation of the 3.14159266 is 11.001001000011111101101010101001000000110110111011
The binary scientific notation representation is 1.1001001000011111101101010101001000000110110111011 x 2^(1)
The IEEE 754 representation is 0|100000000000|1001001000011111101101010101001000000110110111011000

The binary representation of the 1.25 is 1.01
The binary scientific notation representation is 1.01 x 2^(0)
The IEEE 754 representation is 0|11111111111|0100000000000000000000000000000000000000000000000000

The binary representation of the -1.25 is -1.01
The binary scientific notation representation is -1.01 x 2^(0)
The IEEE 754 representation is 1|11111111111|010000000000000000000

In [9]:
len(result)

66

why is it 66 characters and not 64, oh it's cause of the bars